In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose

In [12]:
m = nn.Linear(20,30)

In [13]:
input = torch.ones(128, 20)

In [14]:
output = m(input)

In [15]:
output

tensor([[ 3.2992e-02, -1.3686e+00, -8.9111e-01,  ...,  4.1545e-01,
          1.0630e-01,  4.1503e-04],
        [ 3.2992e-02, -1.3686e+00, -8.9111e-01,  ...,  4.1545e-01,
          1.0630e-01,  4.1503e-04],
        [ 3.2992e-02, -1.3686e+00, -8.9111e-01,  ...,  4.1545e-01,
          1.0630e-01,  4.1503e-04],
        ...,
        [ 3.2992e-02, -1.3686e+00, -8.9111e-01,  ...,  4.1545e-01,
          1.0630e-01,  4.1503e-04],
        [ 3.2992e-02, -1.3686e+00, -8.9111e-01,  ...,  4.1545e-01,
          1.0630e-01,  4.1515e-04],
        [ 3.2992e-02, -1.3686e+00, -8.9111e-01,  ...,  4.1545e-01,
          1.0630e-01,  4.1515e-04]], grad_fn=<AddmmBackward>)